In [1]:
import os
import gc
import warnings
warnings.filterwarnings("ignore")

In [2]:
import evaluate
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from datasets import Dataset, DatasetDict
from transformers import logging, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, PeftConfig, PeftModel
tqdm.pandas()
logging.set_verbosity_error()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
torch.manual_seed(42)

In [ ]:
train_data = pd.read_csv('../dataset/full_train_data_summarization.csv')
train_data_with_title = pd.read_csv('../dataset/full_train_data_title_summarization.csv')
test_data = pd.read_csv('../dataset/full_test_data_summarization.csv')

In [3]:
model_name = 'VietAI/vit5-large'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def preprocess_function(examples):
  inputs = [doc for doc in examples["context"]]
  model_inputs = tokenizer(inputs, max_length=2048, truncation=True, padding=True)
  labels = tokenizer(text_target=examples["summarization"], max_length=768, truncation=True, padding=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [7]:
new_data = DatasetDict({
    "train": Dataset.from_dict(train_data),
    "test": Dataset.from_dict(test_data)
})

In [ ]:
# Map dataset with multiprocessing
tokenized_new_data = new_data.map(preprocess_function, batched=True, num_proc=8)

In [ ]:
# Map dataset not with multiprocessing
tokenized_new_data = new_data.map(preprocess_function, batched=True)

In [9]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)
  decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)
  bleu_metric = evaluate.load("bleu")
  references = [[reference_text] for reference_text in decoded_labels]
  bleu_scores = bleu_metric.compute(references=references, predictions=decoded_preds)
  bleu_score_1 = None
  bleu_score_2 = None
  bleu_score_3 = None
  bleu_score_4 = None
  bleu_score_avg = None
  for k, v in bleu_scores.items():
    if k == "precisions":
      bleu_score_1 = v[0]
      bleu_score_2 = v[1]        
      bleu_score_3 = v[2]        
      bleu_score_4 = v[3]
      bleu_score_avg = (bleu_score_1 + bleu_score_2 + bleu_score_3 + bleu_score_4)/4
      break
  return {
    'bleu@1': bleu_score_1,
    'bleu@2': bleu_score_2,
    'bleu@3': bleu_score_3,
    'bleu@4': bleu_score_4,
    'bleu@avg': bleu_score_avg
  }

In [9]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  rouge_metric = evaluate.load("rouge")
  rouge_scores = rouge_metric.compute(references=decoded_labels, predictions=decoded_preds, use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])
  return {k: round(v, 4) for k, v in rouge_scores.items()}

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto')

In [ ]:
# Config lora for ViT5 Base Model
lora_config = LoraConfig(
  r=8, 
  lora_alpha=16,
  target_modules=["q", "k", "v", "o", "wi", "wo", "lm_head"],
  lora_dropout=0.05,
  bias="none",
  task_type=TaskType.SEQ_2_SEQ_LM
)
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name.replace('/', '_').replace('-', '_')}_model_summarization",
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    # auto_find_batch_size=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    predict_with_generate=True,
    group_by_length=True,
    push_to_hub=False,
    save_total_limit=3,
    report_to='wandb',
    run_name=f'{model_name}',
    save_strategy='epoch',
    evaluation_strategy='no'
)

In [14]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_new_data["train"],
    eval_dataset=tokenized_new_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
torch.cuda.empty_cache()
gc.collect()

40

In [ ]:
trainer.train()

In [16]:
torch.cuda.empty_cache()
gc.collect()
trainer.evaluate()

{'eval_loss': 0.5341997146606445, 'eval_rouge1': 0.3635, 'eval_rouge2': 0.1837, 'eval_rougeL': 0.2778, 'eval_runtime': 1379.3407, 'eval_samples_per_second': 1.595, 'eval_steps_per_second': 0.399, 'epoch': 5.0}


{'eval_loss': 0.5341997146606445,
 'eval_rouge1': 0.3635,
 'eval_rouge2': 0.1837,
 'eval_rougeL': 0.2778,
 'eval_runtime': 1379.3407,
 'eval_samples_per_second': 1.595,
 'eval_steps_per_second': 0.399,
 'epoch': 5.0}

# Test Model Summarization

In [ ]:
peft_model_id = './VietAI_vit5_base_model_summarization/checkpoint-21010/'
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, load_in_8bit=True, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

In [5]:
references = []
predictions = []
def generate_text():
  for batch_1, batch_2 in tqdm(zip(torch.utils.data.DataLoader(test_data['context'], batch_size=16, shuffle=False), torch.utils.data.DataLoader(test_data['summarization'], batch_size=16, shuffle=False), strict=True), total=int(round(len(test_data)/16, 0))):
    encodings = tokenizer(batch_1, max_length=2048, truncation=True, padding=True, return_tensors="pt").to(device)
    outputs = model.generate(
      **encodings,
      max_length=768
    )
    with tokenizer.as_target_tokenizer():
      references.extend(batch_2)
      predictions.extend([tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=False) for output in outputs])
    torch.cuda.empty_cache()

In [ ]:
generate_text()

In [ ]:
test_data[f'generate_vit5'] = test_data['context'].progress_apply(lambda x: generate_text(x))

In [ ]:
test_data[f'generate_vit5'] = predictions

In [7]:
test_data

,context,summarization,generate_vit5
0,"Đa số mọi người tắm mỗi ngày, nhưng thời điểm ...","Tiến sĩ - bác sĩ Jason Singh ở Virginia (Mỹ), ...",Tắm vào buổi tối có nhiều lợi ích hơn. Tắm vào...
1,"Bác sĩ chuyên khoa 1 Hồ Thanh Lịch, Phó khoa H...",Khi đi bơi trong mùa đông cần chú ý lựa chọn t...,Bơi lội là môn thể thao dành cho mọi người ở m...
2,"Bắt đầu ngày mới với tin tức sức khỏe, bạn đọc...",'Một nghiên cứu gần đây của các nhà khoa học M...,Chạy bộ trong mùa đông có nhiều lợi ích. Bơi l...
3,"Theo Bộ Y tế, khu vực Bắc bộ và Bắc Trung bộ x...",Cục Quản lý môi trường y tế (Bộ Y tế) có công ...,Bộ Y tế đề nghị người dân không sử dụng than c...
4,"Năm 2023, thế giới tiếp tục ghi nhận các trườn...","Theo Bộ Y tế, dịch bệnh truyền nhiễm trên thế ...",Bộ Y tế đã tổ chức lễ mít tinh nhân Ngày quốc ...
...,...,...,...
6001,NDRC cho biết chương trình cải cách cơ chế địn...,Chính phủ Trung Quốc đã áp dụng cơ chế điều ch...,Chính phủ Trung Quốc đã áp dụng cơ chế điều ch...
6002,Được xây dựng trên các thảo nguyên khô cằn với...,"Ordos, một thành phố khô cằn ở Trung Quốc, đã ...",Cuộc thi sắc đẹp Ordos đã phát triển mạnh mẽ t...
6003,Tình hình chiến sự thành phố Raqqa tính đến ng...,Tình hình chiến sự thành phố Raqqa tính đến ng...,Lực lượng SDF không tiến lên được và không già...
6004,"Tuy nhiên, cho đến nay diện tích sản xuất lúa ...",Mùa vụ lúa trên đất tôm bắt đầu từ tháng 9 như...,"Ông Phạm Văn Toản, xã Phú Thạnh, huyện Cái Nướ..."


In [ ]:
test_data.to_csv('test_vit5.csv', index=False)

In [5]:
test_vit5 = pd.read_csv('test_vit5_v2.csv')

In [10]:
rouge_metric = evaluate.load("rouge")
rouge_scores = rouge_metric.compute(references=test_vit5['summarization'].tolist(), predictions=test_vit5['generate_vit5'].tolist(), use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])

In [11]:
rouge_scores

{'rouge1': 0.5002105003451729,
 'rouge2': 0.2497110609500507,
 'rougeL': 0.3499058714329288}